# B Refinement Caluclus in Python

Refinement calculus is a technique for building software that's correct-by-construction. One starts with an abstract model of the system or program and continues refining it by adding details to it, e.g, one can add more functionality (called events), one can make data structures more precise (e.g. using functions or arrays instead of sets) to reach a point in which the model is close to code written a traditional programming language such as Java, C, or Python. Typically, writing models in Event-B is conducted with the Rodin platform. The exercise of writing correct-by-construct program requires the proving of proof obligations, which in Event-B is conducted with semi-automated provers, e.g., SMT solvers such as Z3.

**Our goal** is to conduct modeling and proof of Event-B programs with Python, hence, taking advantage of the features that Python provides including the interfacing with Z3, the conection with statics tools and machine learning, and all the underlying libraries.

In [ ]:
!pip install z3-solver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 9.7 MB/s eta 0:00:00


In [ ]:
from z3 import *


# Weakest Precondition calculus



In the style of weakest precondition (WP) calculus, we define Event-B (EB) constructors as property transformers, hence, they take a property and returns the property with apropriate substitutions. WP calculus works backwards from a program post-condition property, and successively uses each program statement to transform the property into the WP property. This is the WP pre-condition in the sense that any other pre-condition that makes the program correct is stronger (implies) the WP.

In [ ]:
# it returns a function that takes a post-condition and 
# replaces free occurrences of x with e
def assg_(x,e):
  return lambda post: \
    substitute(post, (x,e)) # implemented in Z3

# an event body; it replaces in reverse order from the end to the beginning
# notice that in EB the order does not actually matter because 
# assignments are made into disjoint set of variables   
def begin_(*assgs):
    def res(post):
        wp = post
        for s in reversed(assgs):
            wp = s(wp)
        return wp
    return res

# does {pre} prog {post} hold?
# 1. the precondition holds
# 2. the precondition implies the WP
def verify_event(pre,prog,post):
  prove(And(pre,Implies(pre,prog(post))))
 
# does 'pred' hold?
def holds(pred): 
  solver = Solver()
  solver.add(pred)
  res = (solver.check() == sat)
  return res     

# is the event enabled?
def isEnabled(guard): 
    def res(solver):
        solver.push()
        solver.add(guard) 
        enabled = (solver.check() == sat)
        solver.pop() # it guarantees the solver is not modified
        return enabled
    return res

# it runs the event ... Any guard Then body End against post
def event_(guard,body,post): 
    def res(solver):
        if isEnabled(guard)(solver):
            #solver.add(guard)
            weakestPrecond = body(post)
            if weakestPrecond != None:
              solver.add(weakestPrecond)
        return solver
    return res

# skip statement
skip = lambda post: post

# Example: WP calculus

This small example shows how to calculate the WP of a program for some given postcondition <code>postWP</code>.

In [ ]:
x = Const('x',IntSort())
e = IntVal(44)

bodyWP = assg_(x,e)
postWP = (x>IntVal(55))

print('wp: ', bodyWP(postWP))
prove(bodyWP(postWP))

wp:  55 < 44
counterexample
[]


# LTL Syntax in Z3
We show the syntax of our LTL (linear temporal logic) extension (<code>LTL2Py</code>) for safety properties. For additional information about the extension consult paper [Program Synthesis for Cyber-Resilience](https://ieeexplore.ieee.org/document/9760016). 

LTL2Py uses *executed* and *enabled* boolean variables to express LTL properties. Therefore, every time some event <code>e</code> is executed, then <code>e\_executed</code> is set to true; any other *executed* variable is set to false within <code>e</code>'s body.

The syntax of the LTL extension includes non-terminal expressions *LTL*, *LTL*, *Predicate*, and *Events*. Our encoding below uses the <code>Datatype</code> constructed data-type of Z3.


In [ ]:
# datatypes for safety propertes
Ltl = Datatype('Ltl') # LTL datatype
Trace = Datatype('Trace') # Trace datatype
Events = Datatype('Events') # Events datatype

Ltl.declare('after',('after_events',Events),('after_ltl',Ltl))
Ltl.declare('unless',('unless_trace',Trace),('unless_events',Events))
Ltl.declare('trace',('ltl_trace',Trace))

Trace.declare('always', ('always_events',Events))

Events.declare('e_cons', ('e_car',BoolSort()), ('e_cdr',Events))
Events.declare('e_nil')

Trace, Ltl, Events = CreateDatatypes(Trace, Ltl, Events)
after = Ltl.after
unless = Ltl.unless
trace = Ltl.trace
#
after_events = Ltl.after_events
after_ltl = Ltl.after_ltl
ltl_trace = Ltl.ltl_trace
#
unless_trace = Ltl.unless_trace
unless_events = Ltl.unless_events
#
always = Trace.always
always_events = Trace.always_events
#
e_cons = Events.e_cons
e_car = Events.e_car
e_cdr = Events.e_cdr
e_nil = Events.e_nil 

## Encoding LTL properties in Python
Before writting any LTL property, we need to create all the *executed* and *enabled* variables of our model. We use these variables to encode Python LTL extension.

## Example 1: Encoding more LTL properties

We encode the following property using the previous syntax.

```
after send_joining_request_executed (
  always authorize_joining_request_enabled
    unless joining_executed 
)
```  

In [ ]:
send_joining_request_executed = Bool('send_joining_request_executed')
authorize_joining_request_enabled = Bool('authorize_joining_request_enabled')
joining_front_executed = Bool('joining_front_executed')

e1 = (e_cons(send_joining_request_executed,e_nil))
e2 = (e_cons(authorize_joining_request_enabled, e_nil))
e3 = (e_cons(joining_front_executed, e_nil))

trace_prop1 = always(e2)
ltl_prop1 = unless(trace_prop1,e3)
ltl_01 = after(e1,ltl_prop1)

print('LTL property: ')
ltl_01

LTL property: 


after(e_cons(send_joining_request_executed, e_nil),
      unless(always(e_cons(authorize_joining_request_enabled,
                           e_nil)),
             e_cons(joining_front_executed, e_nil)))

```
after joining_executed (
  always authorize_joining_request_disabled
)
``` 

In [ ]:
e4 = (e_cons(authorize_joining_request_enabled == False, e_nil))
e5 = (e_cons(joining_front_executed, e_nil))

trace_prop2 = always(e4)

ltl_prop2 = trace(trace_prop2)
ltl_02 = after(e5,ltl_prop2)

print('LTL property: ')
ltl_02

LTL property: 


after(e_cons(joining_front_executed, e_nil),
      trace(always(e_cons(authorize_joining_request_enabled ==
                          False,
                          e_nil))))

# LTL2Py 
The next function translates <code>LTLPy</code> syntax into Z3 syntax. The translation is a syntactic translation, it's realised with the aid of several *lt2py* functions. The main function is <code>ltl2py_after</code>.

In [ ]:
# It returns a list of boolean assignments: 
# 1. e_executed := BoolVal(True)
# 2. e1_executed := BoolVal(False) for-all e1 in elist
def ltl2py_executed(e, elist):
  varname = e + "_executed"
  globalnames = globals()
  assgs = [assg_(globalnames[varname],BoolVal(True))]
  for e1 in elist:
    if e1 != elist:
      var = e1 + "_executed"
      assgs.append(assg_(globalnames[var],BoolVal(False)))
  return assgs

def ltl2py_event(e):
  return e  

def ltl2py_some_events(el):
  if el == e_nil: return True
  else:
    e = simplify(e_car(el)) 
    l = simplify(e_cdr(el))
    return Or(ltl2py_event(e),ltl2py_some_events(l))

def ltl2py_all_events(el):
  if el == e_nil: 
    return True
  else:
    e = simplify(e_car(el)) 
    l = simplify(e_cdr(el))
    return And(ltl2py_event(e),ltl2py_all_events(l))

def ltl2py_always(a):
  e = simplify(always_events(a))
  return ltl2py_all_events(e)

def ltl2py_trace(t):   
  if Trace.always_events(t) == simplify(Trace.always_events(t)):
    return False
  else:
    e = simplify(always_events(t))
    return ltl2py_all_events(e)

def ltl2py_unless(u):
  e2 = simplify(unless_events(u))
  t = simplify(unless_trace(u))
  return Or(  ltl2py_trace(t),
              Implies(ltl2py_some_events(e2),
                      Not(ltl2py_trace(t)) 
              )
          ) 

def ltl2py_ltl(l):
  if Ltl.unless_trace(l) == simplify(Ltl.unless_trace(l)):
    if Ltl.ltl_trace(l) == simplify(Ltl.ltl_trace(l)): return False
    else: 
      t = simplify(Ltl.ltl_trace(l))
      return ltl2py_trace(t)
  else: 
    return ltl2py_unless(l)

def ltl2py_after(a):
  e1 = simplify(after_events(a))
  l = simplify(after_ltl(a))
  return Implies(ltl2py_all_events(e1), ltl2py_ltl(l))    

## Example 2: translating an LTL property into Z3.
We translate the following property into Z3.

```
after send_joining_request_executed (
  always authorize_joining_request_enabled
    unless joining_front_executed 
)
```  

In [ ]:
send_joining_request_executed = Bool('send_joining_request_executed')
authorize_joining_request_enabled = Bool('authorize_joining_request_enabled')
joining_front_executed = Bool('joining_front_executed')

e1 = (e_cons(send_joining_request_executed,e_nil))
e2 = (e_cons(authorize_joining_request_enabled, e_nil))
e3 = (e_cons(joining_front_executed, e_nil))

trace_prop1 = always(e2)
ltl_prop1 = unless(trace_prop1,e3)
ltl_01 = after(e1,ltl_prop1)

print('LTL property')
print(ltl_01)
print()
inv1 = ltl2py_after(ltl_01)
print('Z3 property')
print(inv1)

LTL property
after(e_cons(send_joining_request_executed, e_nil),
      unless(always(e_cons(authorize_joining_request_enabled,
                           e_nil)),
             e_cons(joining_front_executed, e_nil)))

Z3 property
Implies(And(send_joining_request_executed, True),
        Or(And(authorize_joining_request_enabled, True),
           Implies(Or(joining_front_executed, True),
                   Not(And(authorize_joining_request_enabled,
                           True)))))


```
after joining_front_executed (
  always authorize_joining_request_disabled
)
``` 

In [ ]:
print('LTL property')
print(ltl_02)
print()
inv2 = ltl2py_after(ltl_02)
print('Z3 property')
print(inv2)

print('\nis the Z3 property satisfiable?')

s = Solver()
s.add(inv2)

res = (s.check() == sat)    
if res:
  print('the model is sat')
  print('model: ', s.model())
else:
  print('the mode is unsat: ')
  print('model: ', s) 


LTL property
after(e_cons(joining_front_executed, e_nil),
      trace(always(e_cons(authorize_joining_request_enabled ==
                          False,
                          e_nil))))

Z3 property
Implies(And(joining_front_executed, True),
        And(Not(authorize_joining_request_enabled), True))

is the Z3 property satisfiable?
the model is sat
model:  [joining_front_executed = False,
 authorize_joining_request_enabled = False]



# Platooning: machine variables
For the next parts of this notebook, we model a platooning system in Z3 API of Python, then model some LTL properties, and finally verify them using our WP calculus encoding. The platooning system shown here was firstly modelled in Event-B (a simple and a complex version of it). Here we verify the simple version of the platooning system. We keep the same name conversion used in Event-B: events, machines, machine variables, etc.

We first declare all the (machine) variables and the event names in Z3.

In [ ]:
Z = IntSort()
Position = Datatype('Position')
Position.declare('rear')
Position.declare('front')
Position = Position.create()

# machine variables 
vehiclesSort = SetSort(Z)
vehicles = Const('vehicles',vehiclesSort)
platoon = Const('platoon',vehiclesSort)
next = Array('next', Z, Z)
leader = Const('leader', SetSort(Z))
j_requests = Const('j_requests',vehiclesSort) 
j_authorized = Const('j_authorized',vehiclesSort)
l_requests = Const('j_requests',vehiclesSort) 
l_authorized = Const('j_authorized',vehiclesSort)

# event parameters are created as global machine variables
Vehicle = Const('Vehicle', IntSort())
newVehicle = Const('newVehicle', IntSort())

# *executed* and *enabled* machine variables
add_vehicle_executed = Bool('add_vehicle_executed')
create_platoon_executed = Bool('create_platoon_executed')
set_leader_executed = Bool('set_leader_executed')
send_joining_request_executed = Bool('send_joining_request_executed')
authorize_joining_request_executed = Bool('authorize_joining_request_executed')
joining_front_executed = Bool('joining_front_executed')
joining_front_head_executed = Bool('joining_front_head_executed')
joining_rear_executed = Bool('joining_rear_executed')
joining_rear_tail_executed = Bool('joining_rear_tail_executed')
send_leaving_request_executed = Bool('send_leaving_request_executed')
authorize_leaving_request_executed = Bool('authorize_leaving_request_executed')
leaving_executed = Bool('leaving_executed')

add_vehicle_enabled = Bool('add_vehicle_enabled')
create_platoon_enabled = Bool('create_platoon_enabled')
set_leader_enabled = Bool('set_leader_enabled')
send_joining_request_enabled = Bool('send_joining_request_enabled')
authorize_joining_request_enabled = Bool('authorize_joining_request_enabled')
joining_front_enabled = Bool('joining_front_enabled')
joining_front_head_enabled = Bool('joining_front_head_enabled')
joining_rear_enabled = Bool('joining_rear_enabled')
joining_rear_tail_enabled = Bool('joining_rear_tail_enabled')
send_leaving_request_enabled = Bool('send_leaving_request_enabled')
authorize_leaving_request_enabled = Bool('authorize_leaving_request_enabled')
leaving_enabled = Bool('leaving_enabled')

platoon_event_names = ['add_vehicle','create_platoon', 'set_leader',
         'send_joining_request', 'authorize_joining_request', 
         'joining_front', 'joining_front_head',
         'joining_rear', 'joining_rear_tail',
         'send_leaving_request', 'authorize_leaving_request',
         'leaving']


# Platooning: the machine events
Each machine event is realised through a <code>guard</code> and a <code>body</code> Python function. 

In [ ]:
# machine events
#elist = platoon_event_names

# event: add_vehicle
def guard_add_vehicle(Vehicle):
  return Not(IsMember(Vehicle,vehicles))

def body_add_vehicle(Vehicle,elist) : 
  assgs0 = ltl2py_executed("add_vehicle",elist)
  assgs1 = [assg_(vehicles,SetAdd(vehicles,Vehicle))]
  assgs2 = assgs0+assgs1
  return begin_(*assgs2) 

# event create_platoon
def guard_create_platoon(Vehicle):
  return And(IsMember(Vehicle,vehicles),
             eq(platoon,EmptySet(IntSort())))

def body_create_platoon(Vehicle,elist):
  assgs0 = ltl2py_executed("create_platoon",elist)
  assgs1 = [assg_(platoon,SetAdd(platoon,Vehicle))]
  assgs2 = [assg_(leader,SetAdd(EmptySet(IntSort()),Vehicle))]
  assgs3 = assgs0+assgs1+assgs2
  return begin_(*assgs3)
  
# event: set_leader
def guard_set_leader(Vehicle):
  return IsMember(Vehicle,vehicles)            
  
# event: set_leader
def body_set_leader(Vehicle,elist):
  assgs0 = ltl2py_executed("set_leader",elist)
  assgs1 = [assg_(leader,SetAdd(EmptySet(IntSort()),Vehicle))]
  assgs2 = assgs0+assgs1
  return begin_(*assgs2)

# send_joining_request
def guard_send_joining_request(newVehicle):
  return And(IsMember(newVehicle,vehicles),
             Not(IsMember(newVehicle,platoon)),
             Not(IsMember(newVehicle,j_requests)),
             Not(eq(leader,EmptySet(IntSort()))))
               
def body_send_joining_request(newVehicle,elist):
  assgs0 = ltl2py_executed("send_joining_request",elist)
  assgs1 = [assg_(j_requests,SetAdd(j_requests,newVehicle))]
  assgs2 = assgs0+assgs1
  return begin_(*assgs2) 

# event: authorize_joining_request
def guard_authorize_joining_request(newVehicle):
  return And(IsMember(newVehicle,vehicles),
             Not(IsMember(newVehicle,platoon)),
             IsMember(newVehicle,j_requests),
             Not(eq(leader,EmptySet(IntSort()))))

def body_authorize_joining_request(newVehicle,elist): 
  assgs0 = ltl2py_executed("authorize_joining_request",elist) 
  assgs1 = [assg_(j_requests,SetDel(j_requests,newVehicle)),
           assg_(j_authorized,SetAdd(j_authorized,newVehicle))]
  assgs2 = assgs0+assgs1
  return begin_(*assgs2)


# event: joining_front_head
def guard_joining_front_head(newVehicle,Vehicle):
  return And(IsMember(Vehicle,vehicles),
             IsMember(newVehicle,vehicles),
             Not(IsMember(newVehicle,platoon)),
             IsMember(Vehicle,platoon),
             Not(IsMember(newVehicle,j_requests)),
             IsMember(newVehicle,j_authorized),
             Not(eq(leader,EmptySet(IntSort()))),
             Select(next,Vehicle) == None
  )

def body_joining_front_head(newVehicle,Vehicle,elist):
  assgs0 = ltl2py_executed("joining_front_head",elist)
  assgs1 = [assg_(platoon,SetAdd(platoon,newVehicle)),
            assg_(j_authorized,SetDel(j_authorized,newVehicle)),
            assg_(next,Store(next,Vehicle,newVehicle)),
            ] 
  assgs2 = assgs0+assgs1
  return begin_(*assgs2) 


# event: joining_rear
def guard_joining_rear(newVehicle,Vehicle,Previous):
  return And(IsMember(Vehicle,vehicles),
             IsMember(newVehicle,vehicles),
             IsMember(Previous,vehicles),
             Not(IsMember(newVehicle,platoon)),
             IsMember(Vehicle,platoon),
             IsMember(Previous,platoon),
             Not(IsMember(newVehicle,j_requests)),
             IsMember(newVehicle,j_authorized),
             Not(eq(leader,EmptySet(IntSort()))),
             Select(next,Previous)==Vehicle
  )

def body_joining_rear(newVehicle,Vehicle,Previous,elist):
  assgs0 = ltl2py_executed("joining_rear",elist)
  assgs1 = [assg_(platoon,SetAdd(platoon,newVehicle)),
            assg_(j_authorized,SetDel(j_authorized,newVehicle)),
            assg_(next,Store(Store(next,newVehicle,Select(next,Vehicle)),Vehicle,newVehicle)),
            ] 
  assgs2 = assgs0+assgs1
  return begin_(*assgs2) 

# event: joining_rear_tail
def guard_joining_rear_tail(newVehicle,Vehicle):
  return And(IsMember(Vehicle,vehicles),
             IsMember(newVehicle,vehicles),
             Not(IsMember(newVehicle,platoon)),
             IsMember(Vehicle,platoon),
             Not(IsMember(newVehicle,j_requests)),
             IsMember(newVehicle,j_authorized),
             Not(eq(leader,EmptySet(IntSort()))))
  
def body_joining_rear_tail(newVehicle,Vehicle,elist):
  assgs0 = ltl2py_executed("joining_rear_tail",elist)
  assgs1 = [assg_(platoon,SetAdd(platoon,newVehicle)),
            assg_(j_authorized,SetDel(j_authorized,newVehicle)),
            assg_(next,Store(next,newVehicle,Vehicle)),
            ] 
  assgs2 = assgs0+assgs1
  return begin_(*assgs2) 

# event: leaving
def guard_send_leaving_request(Vehicle):
  return And(IsMember(Vehicle,vehicles),
             IsMember(Vehicle,platoon),
             Not(eq(leader,EmptySet(IntSort()))),
             Not(IsMember(Vehicle,leader))
  )

def body_send_leaving_request(Vehicle,elist):
  assgs0 = ltl2py_executed("send_leaving_request",elist)
  assgs1 = [assg_(l_requests,SetAdd(l_requests,Vehicle))]
  assgs2 = assgs0+assgs1
  return begin_(*assgs2) 

# event: authorize_leaving_request
def guard_authorize_leaving_request(Vehicle):
  return And(IsMember(Vehicle,vehicles),
             IsMember(Vehicle,platoon),
             IsMember(Vehicle,l_requests),
             Not(eq(leader,EmptySet(IntSort()))),
             Not(IsMember(Vehicle,leader))
  )

def body_authorize_leaving_request(Vehicle,elist):
  assgs0 = ltl2py_executed("authorize_leaving_request",elist)  
  assgs1 = [assg_(l_requests,SetDel(l_requests,Vehicle)),
            assg_(l_authorized,SetAdd(l_authorized,Vehicle)),
            assg_(j_requests,SetDel(j_requests,Vehicle)),
            assg_(j_authorized,SetDel(j_authorized,Vehicle))
            ]
  assgs2 = assgs0+assgs1
  return begin_(*assgs2) 

# event: leaving
def guard_leaving(Vehicle): 
  return And(IsMember(Vehicle,vehicles),
             IsMember(Vehicle,platoon),
             IsMember(Vehicle,l_authorized),
             Not(eq(leader,EmptySet(IntSort()))),
             Not(IsMember(Vehicle,leader))
  )

# event: leaving
def body_leaving(Vehicle,elist):  
  assgs0 = ltl2py_executed("leaving",elist)
  assgs1 = [assg_(l_authorized,SetDel(l_authorized,Vehicle)),
            assg_(platoon,SetDel(platoon,Vehicle))
            ]
  assgs2 = assgs0+assgs1
  return begin_(*assgs2) 
  


# Example 4: Testing some of the platooning events

**Test 1**: checks if <code>add_vehicle</code> works properly. 

**Test 2**: check if <code>send_joining_requests</code> works properly.

In [ ]:
vehicles = EmptySet(IntSort())
Vehicle0 = 47
newVehicle0 = 55
elist4 = ['add_vehicle', 'create_platoon', 'set_leader', 'send_joining_request', 
          'authorize_joining_request', 'joining_front_head', 'joining_rear', 'joining_rear_tail', 
          'send_leaving_request', 'authorize_leaving_request', 'leaving']

# test 1

# can we add a Vehicle?
guard0 = guard_add_vehicle(Vehicle0)
#assert(holds(guard0)) # these two lines are
prove(BoolVal(holds(guard0))) #  equivalent

# test 2

# we add Vehicle to vehicles and then check if IsMember(Vehicle,vehicles)
s = Solver()
body0 = body_add_vehicle(Vehicle0,elist4)
post0 = IsMember(Vehicle0,vehicles)
s = event_(guard0,body0,post0)(s)

print("s:", s)

res = (s.check() == sat)    
if res:
  print('test 2: the model is sat')
  print('model: ', s.model())
else:
  print('test 2: the mode is unsat: ')
  print('model: ', s) 


proved
s: [Store(K(Int, False), 47, True)[47]]
test 2: the model is sat
model:  []


# Example 5.a: using WP calculus to verify an LTL formula
We show how to use WP calculus to verifiy the LTL property below for a platooning system. 

```
after send_joining_request_executed (
  always authorize_joining_request_enabled
    unless joining_front_executed 
)
```  

In [ ]:
platoon_event_names = ['add_vehicle','create_platoon', 'set_leader',
         'send_joining_request', 'authorize_joining_request', 
         'joining_front', 'joining_front_head',
         'joining_rear', 'joining_rear_tail',
         'send_leaving_request', 'authorize_leaving_request',
         'leaving']

# *executed* machine variables
add_vehicle_executed = Bool('add_vehicle_executed')
create_platoon_executed = Bool('create_platoon_executed')
set_leader_executed = Bool('set_leader_executed')
send_joining_request_executed = Bool('send_joining_request_executed')
authorize_joining_request_executed = Bool('authorize_joining_request_executed')
joining_front_executed = Bool('joining_front_executed')
joining_front_head_executed = Bool('joining_front_head_executed')
joining_rear_executed = Bool('joining_rear_executed')
joining_rear_tail_executed = Bool('joining_rear_tail_executed')
send_leaving_request_executed = Bool('send_leaving_request_executed')
authorize_leaving_request_executed = Bool('authorize_leaving_request_executed')
leaving_executed = Bool('leaving_executed')

# *enabled* machine variables
add_vehicle_enabled = Bool('add_vehicle_enabled')
create_platoon_enabled = Bool('create_platoon_enabled')
set_leader_enabled = Bool('set_leader_enabled')
send_joining_request_enabled = Bool('send_joining_request_enabled')
authorize_joining_request_enabled = Bool('authorize_joining_request_enabled')
joining_front_enabled = Bool('joining_front_enabled')
joining_front_head_enabled = Bool('joining_front_head_enabled')
joining_rear_enabled = Bool('joining_rear_enabled')
joining_rear_tail_enabled = Bool('joining_rear_tail_enabled')
send_leaving_request_enabled = Bool('send_leaving_request_enabled')
authorize_leaving_request_enabled = Bool('authorize_leaving_request_enabled')
leaving_enabled = Bool('leaving_enabled')

# machine variables 
vehiclesSort = SetSort(IntSort())
vehicles = Const('vehicles',vehiclesSort)
platoon = Const('platoon',vehiclesSort)
next = Array('next', IntSort(), IntSort())
leader = Const('leader', SetSort(IntSort()))
j_requests = Const('j_requests',vehiclesSort) 
j_authorized = Const('j_authorized',vehiclesSort)
l_requests = Const('j_requests',vehiclesSort) 
l_authorized = Const('j_authorized',vehiclesSort)

# *excuted* variables initial values
add_vehicle_executed = BoolVal(False)
create_platoon_executed = BoolVal(False)
set_leader_executed = BoolVal(False)
send_joining_request_executed = BoolVal(False)
authorize_joining_request_executed = BoolVal(False)
joining_front_executed = BoolVal(False)
joining_front_head_executed = BoolVal(False)
joining_rear_executed = BoolVal(False)
joining_rear_tail_executed = BoolVal(False)
send_leaving_request_executed = BoolVal(False)
authorize_leaving_request_executed = BoolVal(False)
leaving_executed = BoolVal(False)

# machine variables initial values
vehicles = EmptySet(IntSort())
platoon = EmptySet(IntSort())
leader = EmptySet(IntSort())
j_requests = EmptySet(IntSort())
j_authorized = EmptySet(IntSort())
l_requests = EmptySet(IntSort())
l_authorized = EmptySet(IntSort())

# event parameters are created as global machine variables
# and not given initial values
Vehicle = Const('Vehicle', IntSort())
newVehicle = Const('newVehicle', IntSort())


# LTL Spec 1
#  after send_joining_request_executed (
#   always authorize_joining_request_enabled
#     unless joining_executed 
#   )
e1 = (e_cons(send_joining_request_executed,e_nil))
e2 = (e_cons(authorize_joining_request_enabled, e_nil))
e3 = (e_cons(joining_rear_tail_executed, e_nil))

alw1 = always(e2)
unl1 = unless(alw1,e3)
ltl_01 = after(e1,unl1)
inv1 = ltl2py_after(ltl_01) 

# "enabled" variables are modelled as system invariants
enb_inv1a = authorize_joining_request_enabled
enb_inv1b = And(IsMember(newVehicle,vehicles),
             Not(IsMember(newVehicle,platoon)),
             IsMember(newVehicle,j_requests),
             Not(eq(leader,EmptySet(IntSort()))))
enb_inv1 = Implies(enb_inv1b,enb_inv1a)

invariant = And(inv1,enb_inv1)

# the precondition
preCond = invariant

# the postcondition
postCond = preCond
prove(postCond)

prog0 = body_authorize_joining_request(newVehicle,platoon_event_names)
wp0 = prog0(postCond)
print('wp0:')
#prove(Implies(wp0,Not(guard_authorize_joining_request(newVehicle))))
prove(Implies(preCond,wp0))

proved
wp0:
proved


# Example 5.b: using WP calculus to verify an LTL formula

```
after send_joining_request_executed (
  always authorize_joining_request_enabled
    unless joining_executed 
)
```  

The Java code below is a unit test for an Eclipse project whose Java code is generated with the <code>EventB2Java</code> tool. The unit test succeeds in Eclipse with JUnit version 4. We replicate below the same unit test but with <code>LTL2Py</code> and using weakest precondition (WP) calculus.

Therefore, we use WP calculus to prove whether the LTL property above holds or not. Notice that, at the style of WP calculus, programs are predicate transformers. Hence, a WP function takes a postcondition and a program (an event implementation) and returns the WP under which the program can be executed and the postcondition will hold. 

WP functions (operators) work from the end to the begining of a program. Thus, in porting the JUnit test into our <code>LTL2Py</code> framework, we succesively calculae the WP for <code>joining_after</code> all the way back to <code>add_vehicle</code>. 

Notice that we need to set all the *enabled* variables to hold whenever the respective event's guard hold. For example, we need to set an invariant for the <code>authorize_joining_request_enabled</code> boolean variable to hold if and only if <code>authorize_joining_request</code>'s event guard does. 

Notice that invariants should hold before and after the execution of every event. Thus, they are added to every event pre- and post-condition. 

Below is the Java unit test. The test succeeds in Eclispe and JUnit version 4.

```java
    @Test
    public void SEND_JOINING_REQUEST_test_inv1() {
    	Integer Vehicle = 0;
    	Integer newVehicle = 1;
    	Integer VehicleLeader = 2;
    	Integer L = platoon.leftLane;
    	Integer X = 0;
    	//
    	ADD_VEHICLE av = new ADD_VEHICLE(machine);
    	assertTrue(av.guard_ADD_VEHICLE(L, Vehicle, X));
    	av.run_ADD_VEHICLE(L, Vehicle, X);
    	assertTrue(machine.get_vehicles().has(Vehicle));
    	assertTrue(av.guard_ADD_VEHICLE(L, newVehicle, X));
    	av.run_ADD_VEHICLE(L, newVehicle, X);
    	assertTrue(machine.get_vehicles().has(newVehicle));
    	assertTrue(av.guard_ADD_VEHICLE(L, VehicleLeader, X));
    	av.run_ADD_VEHICLE(L, VehicleLeader, X);
    	assertTrue(machine.get_vehicles().has(VehicleLeader));
    	//
    	SET_LEADER sl = new SET_LEADER(machine);
    	assertTrue(sl.guard_SET_LEADER(VehicleLeader));
    	sl.run_SET_LEADER(VehicleLeader);
    	assertEquals(VehicleLeader,machine.get_leader());
    	//
    	SEND_JOINING_REQUEST s = new SEND_JOINING_REQUEST(machine);
    	assertTrue(s.guard_SEND_JOINING_REQUEST(newVehicle));
    	s.run_SEND_JOINING_REQUEST(newVehicle);
    	//
    	AUTHORIZE_JOINING_REQUEST a = new AUTHORIZE_JOINING_REQUEST(machine);
    	assertTrue(a.guard_AUTHORIZE_JOINING_REQUEST(VehicleLeader, newVehicle));
    	a.run_AUTHORIZE_JOINING_REQUEST(VehicleLeader, newVehicle);
    	//
    	JOINING_AFTER ja = new JOINING_AFTER(machine);
    	assertTrue(ja.guard_JOINING_AFTER(Vehicle, newVehicle));
    	ja.run_JOINING_AFTER(Vehicle, newVehicle);
    	assertFalse(a.guard_AUTHORIZE_JOINING_REQUEST(VehicleLeader, newVehicle));
    }
```


In [ ]:
platoon_event_names = ['add_vehicle','create_platoon', 'set_leader',
         'send_joining_request', 'authorize_joining_request', 
         'joining_front', 'joining_front_head',
         'joining_rear', 'joining_rear_tail',
         'send_leaving_request', 'authorize_leaving_request',
         'leaving']

# *executed* machine variables
add_vehicle_executed = Bool('add_vehicle_executed')
create_platoon_executed = Bool('create_platoon_executed')
set_leader_executed = Bool('set_leader_executed')
send_joining_request_executed = Bool('send_joining_request_executed')
authorize_joining_request_executed = Bool('authorize_joining_request_executed')
joining_front_executed = Bool('joining_front_executed')
joining_front_head_executed = Bool('joining_front_head_executed')
joining_rear_executed = Bool('joining_rear_executed')
joining_rear_tail_executed = Bool('joining_rear_tail_executed')
send_leaving_request_executed = Bool('send_leaving_request_executed')
authorize_leaving_request_executed = Bool('authorize_leaving_request_executed')
leaving_executed = Bool('leaving_executed')

# *enabled* machine variables
add_vehicle_enabled = Bool('add_vehicle_enabled')
create_platoon_enabled = Bool('create_platoon_enabled')
set_leader_enabled = Bool('set_leader_enabled')
send_joining_request_enabled = Bool('send_joining_request_enabled')
authorize_joining_request_enabled = Bool('authorize_joining_request_enabled')
joining_front_enabled = Bool('joining_front_enabled')
joining_front_head_enabled = Bool('joining_front_head_enabled')
joining_rear_enabled = Bool('joining_rear_enabled')
joining_rear_tail_enabled = Bool('joining_rear_tail_enabled')
send_leaving_request_enabled = Bool('send_leaving_request_enabled')
authorize_leaving_request_enabled = Bool('authorize_leaving_request_enabled')
leaving_enabled = Bool('leaving_enabled')

# machine variables 
vehiclesSort = SetSort(IntSort())
vehicles = Const('vehicles',vehiclesSort)
platoon = Const('platoon',vehiclesSort)
next = Array('next', IntSort(), IntSort())
leader = Const('leader', SetSort(IntSort()))
j_requests = Const('j_requests',vehiclesSort) 
j_authorized = Const('j_authorized',vehiclesSort)
l_requests = Const('j_requests',vehiclesSort) 
l_authorized = Const('j_authorized',vehiclesSort)

# *excuted* variables initial values
add_vehicle_executed = BoolVal(False)
create_platoon_executed = BoolVal(False)
set_leader_executed = BoolVal(False)
send_joining_request_executed = BoolVal(False)
authorize_joining_request_executed = BoolVal(False)
joining_front_executed = BoolVal(False)
joining_front_head_executed = BoolVal(False)
joining_rear_executed = BoolVal(False)
joining_rear_tail_executed = BoolVal(False)
send_leaving_request_executed = BoolVal(False)
authorize_leaving_request_executed = BoolVal(False)
leaving_executed = BoolVal(False)

# *enabled* variables initial values
authorize_joining_request_enabled = BoolVal(False)

# machine variables initial values
# this model works even when the initial values are not the empty-set
vehicles = EmptySet(IntSort())
platoon = EmptySet(IntSort())
leader = EmptySet(IntSort())
j_requests = EmptySet(IntSort())
j_authorized = EmptySet(IntSort())
l_requests = EmptySet(IntSort())
l_authorized = EmptySet(IntSort())

# event parameters are created as global machine variables
# as per the definition of a parameter, they do not have initial values
Vehicle = Const('Vehicle', IntSort())
newVehicle = Const('newVehicle', IntSort())

# LTL Spec 1
#  after send_joining_request_executed (
#   always authorize_joining_request_enabled
#     unless joining_executed 
#   )
e1 = (e_cons(send_joining_request_executed,e_nil))
e2 = (e_cons(authorize_joining_request_enabled, e_nil))
e3 = (e_cons(joining_rear_tail_executed, e_nil))

alw1 = always(e2)
unl1 = unless(alw1,e3)
ltl_01 = after(e1,unl1)
inv1 = ltl2py_after(ltl_01) 

# LTL Spec 2
# after joining_executed (
#  always authorize_joining_request_disabled
# )
e4 = (e_cons(authorize_joining_request_enabled == False, e_nil))
e5 = (e_cons(joining_rear_tail_executed, e_nil))

trace_prop2 = always(e4)

ltl_prop2 = trace(trace_prop2)
ltl_02 = after(e5,ltl_prop2)
inv2 = ltl2py_after(ltl_02)

# LTL Spect 3
# after send_leaving_request executed
#  ( 
# 	always authorize_leaving_request enabled
# 		unless leaving executed
#   )

e6 = (e_cons(send_leaving_request_executed,e_nil))
e7 = (e_cons(authorize_leaving_request_enabled, e_nil))
e8 = (e_cons(leaving_executed, e_nil))
alw3 = always(e7)
unl3 = unless(alw3,e8)
ltl_03 = after(e6,unl3)
inv3 = ltl2py_after(ltl_03) 


# invariants for *enabled* variables: they are logically equivalent
# to their respective event guards
enb_inv1a = authorize_joining_request_enabled
enb_inv1b = And(IsMember(newVehicle,vehicles),
             Not(IsMember(newVehicle,platoon)),
             IsMember(newVehicle,j_requests),
             Not(eq(leader,EmptySet(IntSort()))))
enb_inv1 = And(Implies(enb_inv1b,enb_inv1a),
               Implies(enb_inv1a,enb_inv1b))  


enb_inv2a = authorize_leaving_request_enabled
enb_inv2b = And(IsMember(Vehicle,vehicles),
             IsMember(Vehicle,platoon),
             IsMember(Vehicle,l_requests),
             Not(eq(leader,EmptySet(IntSort()))),
             Not(IsMember(Vehicle,leader)))
enb_inv2 = And(Implies(enb_inv2b,enb_inv2a),
               Implies(enb_inv2a,enb_inv2b))  

# pre: the machine precondition
# the machine precondition should at least count for the machine invariant(s)
preCond = And(inv3,inv2,inv1,enb_inv1,enb_inv2)

# post: the machine postcondition; it should include the machine invariants
postCond = preCond

prove(postCond)

prog0 = body_authorize_joining_request(newVehicle,platoon_event_names)
wp0 = prog0(postCond)
print('wp0:')
prove(Implies(wp0,Not(guard_authorize_joining_request(newVehicle))))

prog1 = body_joining_rear_tail(newVehicle,Vehicle,platoon_event_names)
wp1 = prog1(postCond)
print('\nwp1:')
prove(Implies(wp1,guard_joining_rear_tail(newVehicle,Vehicle)))

prog2 = body_authorize_joining_request(newVehicle,platoon_event_names)
wp2 = prog2(wp1)
print('\nwp2: ')
prove(Implies(wp2,guard_authorize_joining_request(newVehicle)))

prog3 = body_send_joining_request(newVehicle,platoon_event_names)
wp3 = prog3(wp2)
print('\nwp3: ')
prove(Implies(wp3,guard_send_joining_request(newVehicle)))

prog4 = body_create_platoon(Vehicle,platoon_event_names)
wp4 = prog4(wp3)
print('\nwp4: ')
prove(Implies(wp4,guard_create_platoon(Vehicle)))

prog5 = body_add_vehicle(Vehicle,platoon_event_names)
wp5 = prog5(wp4)
print('\nwp5: ')
prove(Implies(wp5,guard_add_vehicle(Vehicle)))

prog6 = body_add_vehicle(newVehicle,platoon_event_names)
wp6 = prog6(wp5)
print('\nwp6: ')
prove(Implies(wp6,guard_add_vehicle(Vehicle)))

counterexample
[authorize_leaving_request_enabled = True]
wp0:
proved

wp1:
proved

wp2: 
proved

wp3: 
proved

wp4: 
proved

wp5: 
proved

wp6: 
proved


We now want to use OCaml (Objective Caml) to express the syntax of our LTL extension instead of relying on Z3's <Datatype>. You can read on how to run OCaml and Python code in a Colab notebook [here](using-python-and-ocaml-in-the-same-jupyter-notebook/). We alsow the syntax of our <code>ltl</code> extension in OCaml.

In [ ]:
!pip install ocaml==0.0.11
!pip install "z3-solver"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ocaml

In [ ]:
%%ocaml
type ltl =
  | After of event * ltl
  (*| Before of (event list) * ltl*)
  (*| Between of (event list) * (event list) * trace*)
  | Until of trace * event * variant * invariant * (event list)
  | Unless of trace * event
  | Ltl of trace 
and trace =
  | Always of predicate
  | Finally of predicate
  | Eventually of predicate * variant * invariant * (event list)
and event =
  | Executed of string
and predicate =
  | Enabled of string
  | Disabled of string
  | AndPred of predicate * predicate
  | OrPred of predicate * predicate
  | NotPred of predicate
  | TruePred
  | FalsePred
and variant =
  | Variant of int
and invariant =
  | Invariant of predicate
  ;;

let rec ltl_pretty_print l =
  match l with
  | After(e,l1) -> "After(" ^event_pretty_print(e)^ ")," ^ltl_pretty_print(l1) ^")"
  | Until(t,e,v,i,m) -> "After(" ^trace_pretty_print(t) ^"," 
                                ^trace_pretty_print(t) ^"," 
                                ^event_pretty_print(e) ^"," 
                                ^variant_pretty_print(v) ^"," 
                                ^invariant_pretty_print(i) ^","
                                ^all_pretty_print(m) ^")"
  | Unless(t,e) -> "Unless(" ^trace_pretty_print(t) ^ ")," ^event_pretty_print(e) ^")"
  | Ltl(t) -> trace_pretty_print(t) 
and trace_pretty_print t =
  match t with 
  | Always(p) -> "Always(" ^pred_pretty_print(p) ^")"
  | Finally(p) -> "Finally(" ^pred_pretty_print(p) ^")"
  | Eventually(p,v,i,elist) -> "Eventually(" ^pred_pretty_print(p) ^")," ^variant_pretty_print(v) ^")," ^invariant_pretty_print(i) ^")," ^some_pretty_print(elist) ^")"
and pred_pretty_print p =
  match p with
  | TruePred -> "BoolVal(True)"
  | FalsePred -> "BoolVal(False)"  
  | NotPred(p) -> "Not(ltl2py_pred(p))" 
  | AndPred(p1,p2) -> "And(ltl2py_pred(p1),ltl2py_pred(p2))"
  | OrPred(p1,p2) -> "Or(ltl2py_pred(p1),ltl2py_pred(p2))"
  | Enabled(e) -> e^"_enabled"
  | Disabled(e) -> "Not(" ^pred_pretty_print(Enabled(e)) ^")"
and all_pretty_print elist =
  match elist with
  | [] -> "BoolVal(True)"
  | e::el -> "And(" ^ event_pretty_print(e) ^"," ^all_pretty_print(el) ^ ")"
and some_pretty_print elist =
  match elist with
  | [] -> "BoolVal(True)"
  | e::el -> "Or("^event_pretty_print(e)^","^some_pretty_print(el)^")"
and event_pretty_print e =
  match e with 
  | Executed(ee) -> ee^"_executed"
and variant_pretty_print v =
  match v with
  | Variant(vv) -> string_of_int(vv)
and invariant_pretty_print i =
  match i with
  | Invariant(p) -> pred_pretty_print(p)    
  ;;  

In [ ]:
%%ocaml
let example_01 =
  let pred1 = Disabled("authorize_joining_request") in
    let trace1 = Always(pred1) in
      let e1 = Executed("joining") in 
        let temporal1 = After(e1,Ltl(trace1)) in
          ltl_pretty_print(temporal1) 
;;

let example_02 =
  let pred1 = Enabled("authorize_joining_request") in
    let trace1 = Always(pred1) in
      let e1 = Executed("joining") in
        let e2 = Executed("send_joining_request") in
          let temporal1 = Unless(trace1,e1) in
            let temporal2 = After(e2,temporal1) in
              ltl_pretty_print(temporal2)
;;

let () = print_endline "LTL safety property in Ocaml:" in
  print_endline example_01
;;

print_endline "";;

let () = print_endline "LTL safety property in Ocaml:" in
  print_endline example_02
;;


LTL safety property in Ocaml:
After(joining_executed),Always(Not(authorize_joining_request_enabled)))

LTL safety property in Ocaml:
After(send_joining_request_executed),Unless(Always(authorize_joining_request_enabled)),joining_executed))


In [ ]:
%%ocaml
let ghost p e =  
  match p, e with
  | Enabled(e1), Executed(e2) -> e1 ^"_enabled_" ^e2 ^"_executed_ghost"
  | Disabled(e1), Executed(e2) -> e1 ^"_disabled_" ^e2 ^"_executed_ghost"
  | _,_ -> "unsupported"
  ;;

let ghost_pred p = 
  match p with
  | Enabled(e1) -> e1 ^"_enabled_ghost"
  | Disabled(e1) -> e1 ^"_disabled_ghost"
  | _ -> "unsupported"
  ;;

let ghost_event e = 
  match e with
  | Executed(e1) -> e1 ^"_executed_ghost"
  ;;


let rec ltl2py_ltl l =
  match l with
  
   (* safety *)
  | After(e1,Ltl(Always(p))) -> "Implies(" ^ltl2py_event(e1) ^")," ^ltl2py_pred(p) ^")" 
  | After(e1,Unless(Always(p),e2)) -> "Implies(" ^ltl2py_event(e1) 
                                                      ^"Or(" ^ltl2py_pred(p) ^")," 
                                                          ^"Implies(" ^ltl2py_event(e2) ^"),"
                                                                    ^ "Not(" ^ltl2py_pred(p) ^")"
                                                              ^")"
                                                      ^")"
                                          ^")" 

   (* liveness *)
   | After(e1,Until(Always(p),e2,Variant(v),Invariant(j),_)) -> 
    "And(" 
    ^"(" ^string_of_int(v) ^")>0,"
    ^"Implies(" ^ghost_event(e1) ^"," ^"And(" ^ltl2py_pred(j) ^"," ^ltl2py_pred(p) ^")),"
    ^"Implies(" ^ltl2py_event(e1) ^"," ^ghost_event(e1) ^"),"
    ^ghost_event(e1) ^"==" ^"Or(oldState(" ^ghost_event(e1) ^")," ^ltl2py_event(e1) ^"),"
    ^"Not(And(" ^string_of_int(v) ^"== 0," ^ghost_event(e1)  ^", Not(" ^ltl2py_event(e2) ^"))),"   
    ^"Implies( And(oldState(" ^ghost_event(e1) ^")," ^"oldState("^ltl2py_pred(j)^")," ^"oldState(" ^ltl2py_pred(p) ^")," ^"Not(" ^ltl2py_event(e2) ^"))," ^string_of_int(v) ^"< oldState(" ^string_of_int(v) ^"))"
    ^")" 
  | After(e,Ltl(Eventually(p,Variant(v),Invariant(j),_))) -> 
    "And(" 
    ^"(" ^string_of_int(v) ^")>0,"
    ^"Implies( And(Not(" ^ghost(p)(e) ^")," ^ltl2py_event(e) ^"), " ^ltl2py_pred(j) ^"),"
    ^"Implies( And(" ^ghost_pred(p) ^"," ^ghost_event(e) ^")," ^ghost(p)(e) ^"),"
    ^"Implies(" ^ltl2py_event(e) ^"," ^ghost_event(e) ^"),"
    ^"Implies( And(" ^ghost_event(e) ^"," ^ltl2py_pred(p)^")," ^ghost(p)(e) ^"),"
    ^"Implies( And(" ^ghost_event(e) ^"," ^ghost_pred(p)^")," ^ghost(p)(e) ^"),"
    ^ghost_event(e) ^"==" ^"Or(oldState(" ^ghost_event(e) ^")," ^ltl2py_event(e) ^"),"
    ^"Or(" ^ghost(p)(e) ^"==" ^"oldState(" ^ghost(p)(e) ^")," ^"And(" ^ghost_event(e) ^ltl2py_pred(p) ^")),"
    ^"Not(And(" ^string_of_int(v) ^"== 0," ^"Not("^ghost(p)(e)^"))),"
    ^"Implies( And(Not(oldState(" ^ghost(p)(e) ^"))," ^"oldState("^ltl2py_event(e)^")," ^ltl2py_pred(p) ^")," ^string_of_int(v) ^"< oldState(" ^string_of_int(v) ^"))"
    ^")" 
  | After(e1,Until(Finally(p),e2,Variant(v),Invariant(j),_)) -> 
    "And("
    ^"(" ^string_of_int(v) ^")>0,"
    ^"Implies( And(Not(" ^ghost(p)(e1) ^")," ^ltl2py_event(e1) ^"), " ^ltl2py_pred(j) ^"),"
    ^"Implies( And(" ^ghost_pred(p) ^"," ^ghost_event(e1) ^")," ^ghost(p)(e1) ^"),"
    ^"Implies(" ^ltl2py_event(e1) ^"," ^ghost_event(e1) ^"),"
    ^ghost_event(e1) ^"==" ^"Or(oldState(" ^ghost_event(e1) ^")," ^ltl2py_event(e1) ^"),"
    ^"Or(" ^ghost(p)(e1) ^"==" ^"oldState(" ^ghost(p)(e1) ^")," ^"And(" ^ghost_event(e1) ^ltl2py_pred(p) ^")),"
    ^"Implies( And(Not(oldState(" ^ghost(p)(e1) ^"))," ^"oldState("^ltl2py_event(e1)^")," ^ltl2py_event(e2) ^")," ^string_of_int(v) ^"< oldState(" ^string_of_int(v) ^"))"
    ^")" 
  | Until(Always(p),e,Variant(v),Invariant(j),_) -> 
    "And("
    ^"(" ^string_of_int(v) ^")>0,"
    ^"Implies( Not(" ^ltl2py_event(e) ^")," ^"And(" ^ltl2py_pred(j) ^"," ^ltl2py_pred(p) ^")),"
    ^"Not(And(" ^string_of_int(v) ^"==0," ^"Not(" ^ltl2py_event(e) ^"))),"
    ^"Implies( And(Not(oldState(" ^ghost_event(e) ^"))," ^"oldState("^ghost_pred(p)^")," ^"Not(" ^ltl2py_event(e) ^"))," ^string_of_int(v) ^"< oldState(" ^string_of_int(v) ^"))"
    ^")" 
  | Until(Finally(p),e,Variant(v),Invariant(j),_) ->  
    "And("
    ^"(" ^string_of_int(v) ^")>0," 
    ^"Implies( Not(" ^ltl2py_event(e) ^")," ^ltl2py_pred(j) ^"),"
    ^"Implies(" ^ltl2py_pred(p) ^"," ^ghost_pred(p) ^"),"
    ^"Implies(oldState(" ^ghost_pred(p) ^")," ^ghost_pred(p) ^"),"
    ^"Not(And(" ^string_of_int(v) ^"==0," ^"Not(" ^ltl2py_event(e) ^"))),"
    ^"Not(And(" ^ltl2py_event(e) ^"," ^"Not(" ^ghost_pred(p) ^"))),"
    ^"Implies( And(oldState(" ^ltl2py_event(e) ^")," ^"oldState(" ^ltl2py_pred(j)  ^"),"  ^"Not(" ^ltl2py_event(e) ^")))"
    ^")" 
  | Ltl(Eventually(p,Variant(v), Invariant(j), _)) ->
    "And("
    ^"(" ^string_of_int(v) ^")>0," 
    ^"Implies( Not(" ^ghost_pred(p) ^")," ^ltl2py_pred(j) ^"),"
    ^"Implies( " ^ltl2py_pred(p) ^"," ^ghost_pred(p) ^"),"
    ^"Implies( oldState(" ^ghost_pred(p) ^")," ^ghost_pred(p) ^"),"
    ^ghost_pred(p) ^"==" ^"Or(oldState(" ^ghost_pred(p) ^")," ^ltl2py_pred(p) ^"),"
    ^"Not(And(" ^string_of_int(v) ^"==0," ^"Not(" ^ghost_pred(p) ^"))),"
    ^"Implies( And(Not(oldState(" ^ghost_pred(p) ^"))," ^"oldState("^ltl2py_pred(j)^")," ^ltl2py_pred(p) ^")," ^string_of_int(v) ^"< oldState(" ^string_of_int(v) ^"))"
    ^")" 
  | Unless(Eventually(p,Variant(v), Invariant(j), _),e) ->  
    "And("
    ^"(" ^string_of_int(v) ^")>0," 
    ^"Implies( Not(" ^ghost_pred(p) ^")," ^ltl2py_pred(j) ^"),"
    ^"Implies( " ^ltl2py_pred(p) ^"," ^ghost_pred(p) ^"),"
    ^"Implies( oldState(" ^ghost_pred(p) ^")," ^ghost_pred(p) ^"),"
    ^ghost_pred(p) ^"==" ^"Or(oldState(" ^ghost_pred(p) ^")," ^ltl2py_pred(p) ^"),"
    ^"Not(And(" ^string_of_int(v) ^"==0," ^"Not(" ^ghost_pred(p) ^"))),"
    ^"Implies( And(Not(oldState(" ^ghost_pred(p) ^"))," ^"oldState("^ltl2py_pred(j)^")," ^"Not(" ^ltl2py_pred(p) ^"))," ^string_of_int(v) ^"< oldState(" ^string_of_int(v) ^"))"
    ^")" 
  | After(e1,Unless(Eventually(p,Variant(v), Invariant(j), _),e2)) ->   
    "And("
    ^"(" ^string_of_int(v) ^")>0," 
    ^"Implies( Not(" ^ltl2py_event(e1) ^")," ^ltl2py_pred(j) ^"),"
    ^"Implies( And(Not(" ^ghost(p)(e1) ^")," ^ltl2py_event(e1) ^")," ^ltl2py_pred(j) ^"),"   
    ^"Implies( And(" ^ghost_event(e1) ^"," ^ghost_pred(p) ^")," ^ghost(p)(e1) ^"),"
    ^"Implies( " ^ltl2py_event(e1) ^"," ^ghost_event(e1) ^"),"
    ^"Implies( And(" ^ghost_event(e1) ^"," ^ltl2py_pred(p) ^")," ^ghost(p)(e1) ^"),"
    ^"Implies( And(" ^ghost_event(e1) ^"," ^ghost_pred(p) ^")," ^ghost(p)(e1) ^"),"
    ^"Implies( " ^ghost_event(e1) ^"==" ^"Or(oldState(" ^ghost_event(e1) ^")," ^ltl2py_event(e1) ^")),"
    ^"Implies( " ^ghost(p)(e1) ^"==" ^"Or(oldState(" ^ghost(p)(e1) ^")," ^"And(" ^ghost_event(e1) ^"," ^ltl2py_pred(p) ^")" ^")),"
    ^"Not(And(" ^string_of_int(v) ^"==0," ^"Not(" ^ghost_pred(p) ^"))),"
    ^"Implies( And(Not(oldState(" ^ghost_event(e1) ^"))," ^"oldState("^ltl2py_pred(j)^")," ^"Not(" ^ltl2py_pred(p) ^"))," ^string_of_int(v) ^"< oldState(" ^string_of_int(v) ^"))"
    ^")" 
  | _ -> "(unsupported)" 
and ltl2py_trace t =
  match t with
  | Always(p) -> "unsupported"
  | Finally(p) -> "Unsupported"
  | Eventually(p,v,i,m) -> "Unsupported"
and ltl2py_pred p =
  match p with
  | TruePred -> "BoolVal(True)"
  | FalsePred -> "BoolVal(False)"  
  | NotPred(p) -> "Not(ltl2py_pred(p))" 
  | AndPred(p1,p2) -> "And(ltl2py_pred(p1),ltl2py_pred(p2))"
  | OrPred(p1,p2) -> "Or(ltl2py_pred(p1),ltl2py_pred(p2))"
  | Enabled(e) -> e^"_enabled"
  | Disabled(e) -> "Not("^ltl2py_pred(Enabled(e))^")"
and ltl2py_event e =
  match e with 
  | Executed(ee) -> ee ^"_executed"
  ;;

**Some examples of Safety and Liveness properties.**

We next pretty-print some safety and liveness properties that we can express in OCaml (Objective Caml).

*Safety*

```
after send_joining_request executed
 ( 
	always authorize_joining_request enabled
		unless joining executed
  )
  ```

In [ ]:
%%ocaml
let example_01 =
 let example = After(Executed("send_joining_request"),
              Unless(Always(Enabled("authorize_joining_request")),Executed("joining_rear"))) in
  ltl_pretty_print example
  ;;


let () = print_endline "property 01:" in
  print_endline example_01
;;

property 01:
After(send_joining_request_executed),Unless(Always(authorize_joining_request_enabled)),joining_rear_executed))


*Safety*

```
after joining executed
 ( 
	always authorize_joining_request disabled
  )
  ```

In [ ]:
%%ocaml
let example_02 =
 let example = After(Executed("joining"),
              Ltl(Always(Disabled("authorize_joining_request")))) in
  ltl_pretty_print example
  ;;


let () = print_endline "property 02:" in
  print_endline example_02
;;

property 02:
After(joining_executed),Always(Not(authorize_joining_request_enabled)))


*Safety*
```
after send_leaving_request executed
 ( 
	always authorize_leaving_request enabled
		unless leaving executed
  )
  ```

In [ ]:
%%ocaml
let example_03 =
 let example = After(Executed("send_leaving_request"),
              Unless(Always(Enabled("authorize_leaving_request")),Executed("joining_rear"))) in
  ltl_pretty_print example
  ;;

let () = print_endline "property 03:" in
  print_endline example_03
;;

property 03:
After(send_leaving_request_executed),Unless(Always(authorize_leaving_request_enabled)),joining_rear_executed))


*Liveness*
```
after vehicle_overtake_request executed
 (
 	eventually vehicle_overtake enabled
  		unless vehicle_change_lane executed 
 )
 ```

In [ ]:
%%ocaml
let example_04 =
 let example = After(Executed("vehicle_overtake_request"),
              Unless(Finally(Enabled("vehicle_overtake")),Executed("vehicle_change_lane"))) in
  ltl_pretty_print example
  ;;

let () = print_endline "property 04:" in
  print_endline example_04
;;

property 04:
After(vehicle_overtake_request_executed),Unless(Finally(vehicle_overtake_enabled)),vehicle_change_lane_executed))


In [ ]:
%%ocaml
let proof_obligations_01 = 
  let v = 5 and i = TruePred and m = [] in
  let example = After(Executed("vehicle_overtake_request"),
                        Unless(Eventually(Enabled("vehicle_overtake"),Variant(v),Invariant(i),m),
                               Executed("vehicle_change_lane"))) in
    ltl_pretty_print example                           
    ;;

let () = print_endline "liveness property: " in
  print_endline proof_obligations_01
  ;;

let proof_obligations_02 = 
  let v = 5 and i = TruePred and m = [] in
  let liveness =  After(Executed("vehicle_overtake_request"),
                        Unless(Eventually(Enabled("vehicle_overtake"),Variant(v),Invariant(i),m),
                               Executed("vehicle_change_lane"))) in
    ltl2py_ltl liveness
    ;;  

let () = print_endline "generated proof obligations: " in
  print_endline proof_obligations_02
  ;;

liveness property: 
After(vehicle_overtake_request_executed),Unless(Eventually(vehicle_overtake_enabled),5),BoolVal(True)),BoolVal(True))),vehicle_change_lane_executed))
generated proof obligations: 
And((5)>0,Implies( Not(vehicle_overtake_request_executed),BoolVal(True)),Implies( And(Not(vehicle_overtake_enabled_vehicle_overtake_request_executed_ghost),vehicle_overtake_request_executed),BoolVal(True)),Implies( And(vehicle_overtake_request_executed_ghost,vehicle_overtake_enabled_ghost),vehicle_overtake_enabled_vehicle_overtake_request_executed_ghost),Implies( vehicle_overtake_request_executed,vehicle_overtake_request_executed_ghost),Implies( And(vehicle_overtake_request_executed_ghost,vehicle_overtake_enabled),vehicle_overtake_enabled_vehicle_overtake_request_executed_ghost),Implies( And(vehicle_overtake_request_executed_ghost,vehicle_overtake_enabled_ghost),vehicle_overtake_enabled_vehicle_overtake_request_executed_ghost),Implies( vehicle_overtake_request_executed_ghost==Or(oldState(v